# NLP with spacy using WML and OpenScale 
Sample Notebook to create a wml deployment based on a python function that uses spacy. Furthermore, the notebook will also create a the OpenScale parts to track this deployment.

In [112]:
import os
import pandas as pd
# we want to show large text snippets to be able to explore the relevant text
pd.options.display.max_colwidth = 400

# Python Function that wraps the Sentiment Prediction

In [113]:
def detect_sentiment():
    import spacy
    from spacy.cli import download
    download('en_core_web_sm')
    nlp = spacy.load("en_core_web_sm")

    def sentiment_analysis(text):
        # Process the text using the Spacy model
        doc = nlp(text)
    
        # Initialize sentiment scores
        positive_score = 0
        negative_score = 0
    
        # Iterate over the tokens in the document
        for token in doc:
            # Check if the token is an adjective
            if token.pos_ == "ADJ":
                # Check if the token has a positive or negative sentiment
                if token.text in ["good", "great", "excellent", "amazing"]:
                    positive_score += 1
                elif token.text in ["bad", "terrible", "awful", "poor"]:
                    negative_score += 1
    
        # Determine the overall sentiment
        if positive_score > negative_score:
            return "SENT_POSITIVE"
        else:
            return "SENT_NEGATIVE"
    
    def score(input):
        
        prediction_values = []
        values = input["input_data"][0]["values"]
        for value in values:
            prediction_values.append([sentiment_analysis(value[0]),[0.5498881125822663, 0.28326629381626844, 0.16684558987617493]])

        scoring_response = {}
        fields = ["prediction","probability"]        
        scoring_response['predictions'] = [
            {
                "fields" : fields,
                "values" : prediction_values
            }
        ]        

        return scoring_response
    
    return score

### Local testing of the python function

In [114]:
scoring_payload = {
   'input_data': [{  
        'fields': ["Text"],
        'values': [
            ["The room is good."],                        
            ["The room is nice, but the price is too expensive. The window frame is made of wood."],
            ["The room is okay."],            
            ["The check-in process is smooth."],
            ["The check-in was horrible."],
            ["The room service is bad."]
        ]
   }]
}

In [115]:
func_result = detect_sentiment()(scoring_payload)
print(func_result)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 103.8 MB/s eta 0:00:00a 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
{'predictions': [{'fields': ['prediction', 'probability'], 'values': [['SENT_POSITIVE', [0.5498881125822663, 0.28326629381626844, 0.16684558987617493]], ['SENT_NEGATIVE', [0.5498881125822663, 0.28326629381626844, 0.16684558987617493]], ['SENT_NEGATIVE', [0.5498881125822663, 0.28326629381626844, 0.16684558987617493]], ['SENT_NEGATIVE', [0.5498881125822663, 0.28326629381626844, 0.16684558987617493]], ['SENT_NEGATIVE', [0.5498881125822663, 0.28326629381626844, 0.16684558987617493]], ['SENT_NEGATIVE', [0.5498881125822663, 0.28326629381626844, 0.16684558987617493]]]}]}


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


{'predictions': [{'fields': ['prediction', 'probability'], 'values': [['SENT_POSITIVE', [0.5498881125822663, 0.28326629381626844, 0.16684558987617493]], ['SENT_NEGATIVE', [0.5498881125822663, 0.28326629381626844, 0.16684558987617493]], ['SENT_NEGATIVE', [0.5498881125822663, 0.28326629381626844, 0.16684558987617493]], ['SENT_NEGATIVE', [0.5498881125822663, 0.28326629381626844, 0.16684558987617493]], ['SENT_NEGATIVE', [0.5498881125822663, 0.28326629381626844, 0.16684558987617493]], ['SENT_NEGATIVE', [0.5498881125822663, 0.28326629381626844, 0.16684558987617493]]]}]}


In [116]:
scoring_payload = {
   'input_data': [{  
        'fields': ["Text"],
        'values': [
            ["Hark, fair Grand Hotel! A wondrous abode, Whose staff so kind did bear a heavy load. Our spacious chamber, with views so divine, Did make us feel as if in Heaven's shrine. Facilities great, with food to delight, Highly recommended, though bath was slight."]
        ]
   }]
}

In [117]:
func_result = detect_sentiment()(scoring_payload)
print(func_result)

  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.7.1/en_core_web_sm-3.7.1-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
{'predictions': [{'fields': ['prediction', 'probability'], 'values': [['SENT_POSITIVE', [0.5498881125822663, 0.28326629381626844, 0.16684558987617493]]]}]}


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


{'predictions': [{'fields': ['prediction', 'probability'], 'values': [['SENT_POSITIVE', [0.5498881125822663, 0.28326629381626844, 0.16684558987617493]]]}]}


# Deploying the Python Function to IBM Watson Machine Learning

In [118]:
#masked
WOS_CREDENTIALS = {
    "url": "https://xxx/",
    "username": "xxx",
    "password": "xxx"
}

WML_CREDENTIALS = WOS_CREDENTIALS.copy()
WML_CREDENTIALS["instance_id"] = "wml_local"
WML_CREDENTIALS["version"] = "5.0" #If your env is CP4D 4.x.x then specify "4.x.x" instead of "4.6"

# The space where the WML model is deployed. Please note the model deployment is not actually used at runtime by OpenScale, but used only to generate the OpenScale monitor artifacts.
space_uid = '0c3328b8-d508-40e0-880c-89a757aa5630'

# insert Service instance id from OpenScale - default is 00000000-0000-0000-0000-000000000000
service_instance_id = data_mart_id = "00000000-0000-0000-0000-000000000000"

In [119]:
from ibm_watson_machine_learning import APIClient
wml_client = APIClient(WML_CREDENTIALS)

ConnectionError: HTTPSConnectionPool(host='xxx', port=443): Max retries exceeded with url: /ml/wml_services/version (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f3f26d9b350>: Failed to establish a new connection: [Errno -2] Name or service not known'))

In [ ]:
wml_client.set.default_space(space_uid)

In [ ]:
PYTHON_FUNCTION_NAME = 'spacy_sentiment_function'
DEPLOYMENT_NAME = "spacy-nlp-sentiment-deploy"

#### Python Function cleanup

In [ ]:
deployments_list = wml_client.deployments.get_details()
for deployment in deployments_list["resources"]:
    model_id = deployment["entity"]["asset"]["id"]
    deployment_id = deployment["metadata"]["id"]
    if deployment["metadata"]["name"] == DEPLOYMENT_NAME:
        print("Deleting deployment id", deployment_id)
        wml_client.deployments.delete(deployment_id)
        time.sleep(5)
        print("Deleting model id", model_id)
        wml_client.repository.delete(model_id)

wml_client.repository.list_functions()

## Store the python function

In [ ]:
pyfunc_swspec_id = wml_client.software_specifications.get_uid_by_name("NLP-Sandbox")

meta_data = {
    wml_client.repository.FunctionMetaNames.NAME: PYTHON_FUNCTION_NAME,
    wml_client.repository.FunctionMetaNames.DESCRIPTION: PYTHON_FUNCTION_NAME,
    wml_client.repository.FunctionMetaNames.SOFTWARE_SPEC_UID: pyfunc_swspec_id
}


sentiment_function_details = wml_client.repository.store_function(meta_props=meta_data, function=detect_sentiment)

## Deploy the python function

In [ ]:
sentiment_function_uid = wml_client.repository.get_function_uid(sentiment_function_details)

meta_props = {
   wml_client.deployments.ConfigurationMetaNames.NAME: DEPLOYMENT_NAME,
   wml_client.deployments.ConfigurationMetaNames.DESCRIPTION: DEPLOYMENT_NAME,
   wml_client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: { 'name': 'M'},  
   wml_client.deployments.ConfigurationMetaNames.ONLINE: {   }
}

sentiment_deployment_details = wml_client.deployments.create(sentiment_function_uid, meta_props=meta_props)

.

.

.

.

.

...

.

.

.

.


ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='76007d86-0c07-4a8e-a571-a034abedc411'
------------------------------------------------------------------------------------------------




In [ ]:
sentiment_deployment_id = wml_client.deployments.get_uid(sentiment_deployment_details)

In [ ]:
function_details = wml_client.repository.get_details(sentiment_function_uid)
from pprint import pprint
pprint(function_details)

## Get the scoring endpoint for the deployed python function

In [ ]:
scoring_url = wml_client.deployments.get_scoring_href(sentiment_deployment_details)

## Scoring against the deployed function

In [ ]:
predictions = wml_client.deployments.score(sentiment_deployment_id, scoring_payload)
predictions

# IBM Watson OpenScale

In [78]:
from ibm_cloud_sdk_core.authenticators import CloudPakForDataAuthenticator
from ibm_watson_openscale import APIClient

from ibm_watson_openscale import *
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.supporting_classes import *
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import *

In [79]:
from ibm_cloud_sdk_core.authenticators import CloudPakForDataAuthenticator
from ibm_watson_openscale import APIClient as WOSAPIClient

wos_client = WOSAPIClient(
authenticator = CloudPakForDataAuthenticator(
        url=WML_CREDENTIALS['url'],
        username=WML_CREDENTIALS['username'],
        password=WML_CREDENTIALS['password'],
        disable_ssl_verification=True
    ),
    service_url=WML_CREDENTIALS['url'],
    service_instance_id=service_instance_id
)

print(wos_client.version)

3.0.39


In [80]:
# Listing service providers
wos_client.service_providers.show()

99999999-9999-9999-9999-999999999999,active,service-provider-space-0c3328b8-d508-40e0-880c-89a757aa5630,watson_machine_learning,2024-11-18 14:18:03.325000+00:00,efa2860a-0d50-4704-873e-5aae2e39f56c
99999999-9999-9999-9999-999999999999,active,WOS ExpressPath WML pre_production binding,watson_machine_learning,2024-11-18 11:02:02.245000+00:00,f1161d5f-0bfe-48b1-b3c7-d07bd00963d3
99999999-9999-9999-9999-999999999999,active,WOS ExpressPath WML production binding,watson_machine_learning,2024-11-18 11:01:56.876000+00:00,bb60b308-8909-454a-a0de-87400c0c665c


In [81]:
# Copy the ID of your service provider from the `id` column in the output of the cell above
SERVICE_PROVIDER_ID = "efa2860a-0d50-4704-873e-5aae2e39f56c"

In [82]:
# Remove existing credit risk subscription
subscriptions = wos_client.subscriptions.list().result.subscriptions
for subscription in subscriptions:
    sub_model_id = subscription.entity.asset.asset_id
    if sub_model_id == sentiment_function_uid:
        wos_client.subscriptions.delete(subscription.metadata.id)
        print('Deleted existing subscription for model', sub_model_id)

In [83]:
# Show asset
asset_deployment_details = wos_client.service_providers.list_assets(data_mart_id=data_mart_id, 
    service_provider_id=SERVICE_PROVIDER_ID, deployment_id = sentiment_deployment_id, deployment_space_id=space_uid).result['resources'][0]
asset_deployment_details

{'metadata': {'guid': '76007d86-0c07-4a8e-a571-a034abedc411',
  'created_at': '2024-11-19T13:57:17.143Z',
  'modified_at': '2024-11-19T13:57:17.143Z'},
 'entity': {'name': 'spacy-nlp-sentiment-deploy',
  'type': 'online',
  'description': 'spacy-nlp-sentiment-deploy',
  'scoring_endpoint': {'url': 'https://internal-nginx-svc:12443/ml/v4/deployments/76007d86-0c07-4a8e-a571-a034abedc411/predictions'},
  'asset': {},
  'asset_properties': {}}}

In [84]:
# Show model
model_asset_details_from_deployment=wos_client.service_providers.get_deployment_asset(data_mart_id=data_mart_id,
    service_provider_id=SERVICE_PROVIDER_ID, deployment_id=sentiment_deployment_id, deployment_space_id=space_uid)
model_asset_details_from_deployment

{'metadata': {'guid': '76007d86-0c07-4a8e-a571-a034abedc411',
  'created_at': '2024-11-19T13:57:17.143Z',
  'modified_at': '2024-11-19T13:57:17.143Z'},
 'entity': {'name': 'spacy-nlp-sentiment-deploy',
  'type': 'online',
  'description': 'spacy-nlp-sentiment-deploy',
  'scoring_endpoint': {'url': 'https://internal-nginx-svc:12443/ml/v4/deployments/76007d86-0c07-4a8e-a571-a034abedc411/predictions'},
  'asset': {'asset_id': '393676a7-249d-4a08-9116-dc5764b60b94',
   'url': 'https://internal-nginx-svc:12443/ml/v4/functions/393676a7-249d-4a08-9116-dc5764b60b94?space_id=0c3328b8-d508-40e0-880c-89a757aa5630&version=2020-06-12',
   'name': 'spacy_sentiment_function',
   'asset_type': 'function',
   'created_at': '2024-11-19T13:57:15.341Z',
   'modified_at': '2024-11-19T13:57:16.265Z'},
  'asset_properties': {'model_type': 'python',
   'runtime_environment': 'python-3.11'}}}

In [85]:
#Create subscription
subscription_details = wos_client.subscriptions.add(
        data_mart_id=data_mart_id,
        background_mode=False,
        service_provider_id=SERVICE_PROVIDER_ID,
        asset=Asset(
            asset_id=model_asset_details_from_deployment["entity"]["asset"]["asset_id"],
            name=model_asset_details_from_deployment["entity"]["asset"]["name"],
            url=model_asset_details_from_deployment["entity"]["asset"]["url"],
            asset_type=AssetTypes.FUNCTION,
            input_data_type=InputDataType.UNSTRUCTURED_TEXT,
            problem_type=ProblemType.MULTICLASS_CLASSIFICATION
        ),
        deployment=AssetDeploymentRequest(
            deployment_id=asset_deployment_details['metadata']['guid'],
            name=asset_deployment_details['entity']['name'],
            deployment_type= DeploymentTypes.ONLINE,
            url=asset_deployment_details['entity']['scoring_endpoint']['url']
        ),
        asset_properties=AssetPropertiesRequest(
            label_column='label',
            probability_fields=["probability"],
            prediction_field="prediction",
            feature_fields = ["Text"],
            categorical_fields = ["Text"]
        )
    ).result

subscription_id = subscription_details.metadata.id
subscription_id




 Waiting for end of adding subscription fbcfded5-1454-41a4-ab52-0c0e0e64755e 






preparing

.


active

-------------------------------------------
 Successfully finished adding subscription 
-------------------------------------------




'fbcfded5-1454-41a4-ab52-0c0e0e64755e'

### Get the Payload logging data set id

In [86]:
import time

time.sleep(5)
payload_data_set_id = None
payload_data_set_id = wos_client.data_sets.list(type=DataSetTypes.PAYLOAD_LOGGING, 
                                                target_target_id=subscription_id, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result.data_sets[0].metadata.id
if payload_data_set_id is None:
    print("Payload data set not found. Please check subscription status.")
else:
    print("Payload data set id: ", payload_data_set_id)

Payload data set id:  7c1941d9-e11e-4779-bc4d-c60ae982c1db


### Score against the model again and check if the payload was logged

In [110]:
predictions = wml_client.deployments.score(sentiment_deployment_id, scoring_payload)
predictions

{'predictions': [{'fields': ['prediction', 'probability'],
   'values': [['SENT_POSITIVE',
     [0.5498881125822663, 0.28326629381626844, 0.16684558987617493]]]}]}

In [111]:
time.sleep(10)
pl_records_count = wos_client.data_sets.get_records_count(payload_data_set_id)
print("Number of records in the payload logging table: {}".format(pl_records_count))

Number of records in the payload logging table: 3


# IBM Watson OpenScale - Explainability

In [89]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)
parameters = {
    "enabled": True,
    "perturbations_count": 100
}
explainability_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.EXPLAINABILITY.ID,
    target=target,
    parameters=parameters
).result

explainability_monitor_id = explainability_details.metadata.id




 Waiting for end of monitor instance creation 752fb7ce-ee63-451e-aafd-f01a8d860f5d 






preparing


active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




## Sample records for which explainability needs to run

In [90]:
payload_data = wos_client.data_sets.get_list_of_records(data_set_id=payload_data_set_id,output_type='pandas').result
explanation_types = ["lime"]

scoring_ids = payload_data.head(1)['scoring_id'].tolist()
print(scoring_ids)

['1f529170333bc5408d37c6e8c4a57321-1']


## Run explainability

In [91]:
import sys,os,os.path
import json
import requests
import base64
from requests.auth import HTTPBasicAuth
import time

In [92]:
headers = {}
headers["Content-Type"] = "application/json"
headers["Authorization"] = "Bearer {}".format(os.environ['USER_ACCESS_TOKEN'])

### Explain payload

In [93]:
explanations_tasks_payload = {
  "scoring_ids": scoring_ids,
  "subscription_id": subscription_id,
  "explanation_types": ["lime"],
  "parameters": {
       "lime": {
          "perturbations_count": 100
       }
   }
}

### Submit the explain task

In [94]:
explanations_tasks_url =  WML_CREDENTIALS['url'] + "/openscale/{0}/v2/explanation_tasks".format(data_mart_id)
response = requests.post(explanations_tasks_url, json=explanations_tasks_payload, headers=headers, verify=False)
json_data = response.json()
print(json_data)

{'metadata': {'explanation_task_ids': ['58d9400f-1894-4305-890b-95c66ad5da5a'], 'created_at': '2024-11-19T13:59:48.806697Z', 'created_by': '1000330999'}}


In [95]:
explanation_task_ids = json_data['metadata']['explanation_task_ids']
explanation_task_ids

['58d9400f-1894-4305-890b-95c66ad5da5a']

### Check for the status of the explain run.

In [96]:
def finish_explanation_tasks(sample_size = 1):
    finished_explanations = []
    finished_explanation_task_ids = []
    
    # Check for the explanation task status for finished status. 
    # If it is in-progress state, then sleep for some time and check again. 
    # Perform the same for couple of times, so that all tasks get into finished state.
    for i in range(0, 15):
        # for each explanation
        print('iteration ' + str(i))
        
        #check status for all explanation tasks
        for explanation_task_id in explanation_task_ids:
            if explanation_task_id not in finished_explanation_task_ids:
                result = wos_client.monitor_instances.get_explanation_tasks(explanation_task_id=explanation_task_id, subscription_id=subscription_id ).result
                print(explanation_task_id + ' : ' + result.entity.status.state)
                if (result.entity.status.state == 'finished' or result.entity.status.state == 'error') and explanation_task_id not in finished_explanation_task_ids:
                    finished_explanation_task_ids.append(explanation_task_id)
                    finished_explanations.append(result)


        # if there is altest one explanation task that is not yet completed, then sleep for sometime, 
        # and check for all those tasks, for which explanation is not yet completeed.
        
        if len(finished_explanation_task_ids) != sample_size:
            print('sleeping for some time..')
            time.sleep(10)
        else:
            break
                    
    return finished_explanations

In [97]:
finished_explanations = finish_explanation_tasks(1)

iteration 0


58d9400f-1894-4305-890b-95c66ad5da5a : in_progress
sleeping for some time..


iteration 1


58d9400f-1894-4305-890b-95c66ad5da5a : in_progress
sleeping for some time..


iteration 2


...

58d9400f-1894-4305-890b-95c66ad5da5a : in_progress
sleeping for some time..


iteration 9


58d9400f-1894-4305-890b-95c66ad5da5a : in_progress
sleeping for some time..


iteration 10


58d9400f-1894-4305-890b-95c66ad5da5a : finished


### Print the explain result

In [98]:
for result in finished_explanations:
    print(result)

{
  "metadata": {
    "explanation_task_id": "58d9400f-1894-4305-890b-95c66ad5da5a",
    "created_by": "1000330999",
    "created_at": "2024-11-19T13:59:48.806697Z",
    "updated_at": "2024-11-19T14:01:34.339867Z"
  },
  "entity": {
    "status": {
      "state": "finished"
    },
    "asset": {
      "id": "393676a7-249d-4a08-9116-dc5764b60b94",
      "name": "spacy_sentiment_function",
      "input_data_type": "unstructured_text",
      "problem_type": "multiclass",
      "deployment": {
        "id": "76007d86-0c07-4a8e-a571-a034abedc411",
        "name": "spacy-nlp-sentiment-deploy"
      }
    },
    "input_features": [
      {
        "name": "Text",
        "value": "Hark, fair Grand Hotel! A wondrous abode, Whose staff so kind did bear a heavy load. Our spacious chamber, with views so divine, Did make us feel as if in Heaven's shrine. Facilities great, with food to delight, Highly recommended, though bath was slight."
      }
    ],
    "perturbed": false,
    "explanations": [

# IBM Watson OpenScale - Quality Monitoring

## Quality monitor configuration

In [99]:
import time

#time.sleep(10)
target = Target(
        target_type=TargetTypes.SUBSCRIPTION,
        target_id=subscription_id
)
parameters = {
    "min_feedback_data_size": 100
}
thresholds = [
                {
                    "metric_id": "accuracy",
                    "type": "lower_limit",
                    "value": .90
                },
                {
                    "metric_id": "weighted_true_positive_rate",
                    "type": "lower_limit",
                    "value": .90
                },
                {
                    "metric_id": "weighted_precision",
                    "type": "lower_limit",
                    "value": .90
                },
                {
                    "metric_id": "log_loss",
                    "type": "upper_limit",
                    "value": .90
                },
                {
                    "metric_id": "weighted_recall",
                    "type": "lower_limit",
                    "value": .90
                },
                {
                    "metric_id": "weighted_f_measure",
                    "type": "lower_limit",
                    "value": .90
                }
,
                {
                    "metric_id": "weighted_false_positive_rate",
                    "type": "upper_limit",
                    "value": .20
                }    
    
            ]
quality_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.QUALITY.ID,
    target=target,
    parameters=parameters,
    thresholds=thresholds
).result




 Waiting for end of monitor instance creation 542afcfd-1c5a-4339-a0a3-985d1fbe0856 






preparing


active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




In [100]:
quality_monitor_instance_id = quality_monitor_details.metadata.id
quality_monitor_instance_id

'542afcfd-1c5a-4339-a0a3-985d1fbe0856'

### Get the feedback data set id

In [101]:
feedback_dataset_id = None
feedback_dataset = wos_client.data_sets.list(type=DataSetTypes.FEEDBACK, 
                                                target_target_id=subscription_id, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result
feedback_dataset_id = feedback_dataset.data_sets[0].metadata.id
if feedback_dataset_id is None:
    print("Feedback data set not found. Please check quality monitor status.")

In [102]:
feedback_dataset_id

'990d33e0-6228-4e24-ac15-d3e0329fab1c'

## Load feedback data to feedback logging table

In [103]:
import pandas as pd
feed_data_load = pd.read_csv('https://raw.githubusercontent.com/ravichamarthy/watson-nlp/main/tripadvisor_hotel_reviews_feedback_data_100_records.csv')
feedback_data = json.loads(feed_data_load.to_json(orient='records'))

In [104]:
wos_client.data_sets.store_records(feedback_dataset_id, request_body=feedback_data, background_mode=False)




 Waiting for end of storing records with request id: 0ecb3924-eb50-432c-8c94-98fe1804b019 






pending


active

---------------------------------------
 Successfully finished storing records 
---------------------------------------




In [105]:
wos_client.data_sets.show_records(data_set_id=feedback_dataset_id)

bda02f41-f06c-440b-8c20-f2c5877903e8,2024-11-19T14:02:03.385Z,SENT_NEGATIVE,None,"old dumpy place problems head starts spinning thinking start, got like 10pm check-in told park street cuz no parking left lot, room tiny carpet dirty spots, room came kitchen nasty used, especially fridge time opened nasty odor rotten vegetables gushed fridge filled entire room, day night hear people talking walking stairwell right outside, noise streets right outside window substantial, just no sound insulation place, bathroom surprisingly clean room appeared well-kept, shower pressure horrible stay ran hot water twice, fact morning took cold shower basically start end, desk staff generally nice, guy completely uncaring unhelpful not sure problem, thing liked place ceiling fan walk-in closet, paid 100/night place suppose reasonable seattle center area, pay 40 night stay holiday inn express, basically ca n't recommend place,"
18b63a71-1f27-4b2e-a089-be054140ab34,2024-11-19T14:02:03.384Z,SENT_NEGATIVE,None,"terrible hotel approximately 2 weeks ago april 25 2007 reservation hotel night 19 2007, wrote follow-up request reply advised reservation cancelled overbooking.i wote giving reservation number just wrote just saying sorry making no effort room nearby hotel, rhe emails brigeta operation manager, not written approximately 2 weeks stay positive left scrambe new hotel lobby inn queen anne.i practice unforgivable especially light having resevation number guarenteeing room 19 2007. point making advance reservations hotel suddenly cancel little advance notice sincerely greg scanlonsan francisco,"
1a2ed078-6b46-4ea6-9a7c-c32529b73892,2024-11-19T14:02:03.384Z,SENT_POSITIVE,None,"loved inn queen anne really great stay, great staff continental breakfast locationsuite included bedroom lounge room kitchen 2 walk-in robes den,"
224fcedf-de30-4cc3-852d-8aa0bab607e6,2024-11-19T14:02:03.384Z,SENT_POSITIVE,None,"return going seattle booked hotel knowing budget hotel, check-in friends met friendly warm welcome, rooms read reviews simple charming, beds-ok bathroom-ok really big closet nice little kitchenette.the breakfast not described great experience needs little kitchenette, bring coffee lobby prepare bacon eggs bought supermarket located not 150 metres away.what stay great experience friendly staff especially dana conveniant compementary shuttle service low price great location, 10-15 minute walk city center 2 minutes key arena 20 minutes hooters restaurant 20 minutes pike market 8 minutes space needle.///anders friends,"
6bd638ea-e635-4d20-8473-275cb83d6e02,2024-11-19T14:02:03.384Z,SENT_POSITIVE,None,"good experience having booked minute escape university tower hotel review proved excellent minute find.central clean rooms good views albeit rooms exec levels n't exactly spacious.try book executive rooms possible good choice breakfast hors d'oeuvres day makes overall package palatable.restaurant little pricey good food staying night skip try benihana just road hotel great food atmosphere,"
78a02ad9-bbda-4b76-a498-0e2948a4e091,2024-11-19T14:02:03.384Z,SENT_POSITIVE,None,"not bad place stay short term n't expect, want roominess amenities not walk stairs not place you.if day just need place sleep shower fine place, rate good compared lodgings city.it old 1920 built twin apartment buildings retired priests nuns, building sisters priests.our room queen bed large walk-in closet kitchenette bath, clean adequate night stay.the staff friendly bellboy..adam nice helpful, stairway funny smell mildew like room no air conditioner no smell, temps hit 90 open windows ceiling fan bearable, hot temperature seattle rare.this street city center old 1962 worlds fair site space needle park amusement rides short walk away, monorail 2 ride downtown, walk free public transportations pioneer square pike markey waterfront.if arriving air rush-hour period suggest let hotel book towncar, 45 plus tip taxi ride rush-hour expensive, stuck traffic nice not meter, went taxi 5 a.m. sunday cost 

In [106]:
time.sleep(5)
wos_client.data_sets.get_records_count(feedback_dataset_id)

100

## Run Quality Monitor

In [107]:
run_details = wos_client.monitor_instances.run(monitor_instance_id=quality_monitor_instance_id, background_mode=False).result




 Waiting for end of monitoring run 585053ae-951f-4f93-af67-603e03e460a0 






running


finished

---------------------------
 Successfully finished run 
---------------------------




## Fetch the Quality Metrics

In [108]:
#time.sleep(5)
wos_client.monitor_instances.show_metrics(monitor_instance_id=quality_monitor_instance_id)

Author: Ravi Chamarthy (ravi.chamarthy@in.ibm.com) - adjusted by Tammo Heuzeroth and Benedikt Bothur